In [ ]:
pip install tokenizers

In [ ]:
pip install transformers

In [ ]:
import string

train_file = open("train.fo1.txt", "r", encoding="utf-8")


train_sents = []
for i in train_file:
  train_sents.append(i)
train_data  = " "

In [ ]:
from tokenizers import ByteLevelBPETokenizer

In [ ]:
tokenizer = ByteLevelBPETokenizer()

tokenizer.train("train.fo1.txt", vocab_size = 33_278, min_frequency = 2, 
                special_tokens = ['<s>', '<pad>', '</s>', '<unk>', '<mask>'])

In [ ]:
tokenizer.save_model('tokenizer')

In [ ]:
from transformers import RobertaTokenizer

In [ ]:
#config_file = RobertaTokenizer.from_pretrained('tokenizer')
tokenizer = RobertaTokenizer.from_pretrained('tokenizer', max_len=512)

In [ ]:
tokens = tokenizer('as')
tokens

In [ ]:
#@title 預設標題文字
#@title labels == input_ids
#@title input_ids -> MLM

In [ ]:
import torch
#15% of the words are being masked
def mLm(tensor):
  rand = torch.rand(tensor.shape)
  mask_arr = (rand < 0.15) * (tensor > 2)
  for i in range(tensor.shape[0]):
    selection = torch.flatten(mask_arr[i].nonzero()).tolist()
    tensor[i, selection] = 4
  return tensor


In [ ]:
input_ids = []
mask = []
labels = []

train_sents = []
with open("train.fo1.txt", "r", encoding = "utf8") as file:
  sents = file.read().split('\n')
  sample = tokenizer(sents, max_length=512, padding='max_length', truncation=True,return_tensors='pt')
  labels.append(sample.input_ids)
  mask.append(sample.attention_mask)
  input_ids.append(mLm(sample.input_ids.detach().clone()))




In [ ]:
input_ids = torch.cat(input_ids)
mask = torch.cat(mask)
labels = torch.cat(labels)

In [ ]:
input_ids[0][:10]
labels[0][:10]

tensor([    0,  3221, 27251,   546,  7757,   474,   539,   225,     3,  7775])

In [ ]:
encodings = {
    'input_ids': input_ids,
    'attention_mask': mask,
    'labels': labels
}

In [ ]:
class Dataset(torch.utils.data.Dataset):
  def __init__(self, encodings):
    self.encodings = encodings
  def __len__(self):
    return self.encodings['input_ids'].shape[0]
  def __getitem__(self, i):
    return { key: tensor[i] for key, tensor in self.encodings.items()}

In [ ]:
dataset = Dataset(encodings)

In [ ]:
dataloader = torch.utils.data.DataLoader(dataset, batch_size = 16, shuffle = True)

In [ ]:
from transformers import RobertaConfig

In [ ]:
config = RobertaConfig(
    vocab_size = 33_278,
    max_position_embeddings = 514,
    hidden_size = 768,
    num_attention_heads = 12,
    num_hidden_layers = 6,
    type_vocab_size = 1
)

In [ ]:
from transformers import RobertaForMaskedLM

In [ ]:
model = RobertaForMaskedLM(config)

In [ ]:
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

In [ ]:
model.to(device)

In [ ]:
from transformers import AdamW

In [ ]:
model.train()

In [ ]:
optim = AdamW(model.parameters(), lr=1e-6)

In [ ]:
import torch.nn as nn
criterion = nn.CrossEntropyLoss()

In [ ]:
epochs = 2
step = 0

In [ ]:
from tqdm import tqdm

#loop = tqdm(enumerate(iter(dataloader), 0), leave=True)

for epoch in range(epochs):
  loop = tqdm(dataloader, leave= True)
  for batch in loop:
    optim.zero_grad()
    input_ids = batch['input_ids'].to(device)
    mask = batch['attention_mask'].to(device)
    labels = batch['labels'].to(device)
    outputs = model(input_ids, attention_mask = mask, labels=labels)

  #loss = criterion(outputs, labels)
    loss = outputs.loss
    loss.backward()
    optim.step()

    loop.set_description(f'Epoch {epoch}')
    loop.set_postfix(loss = loss.item())

In [ ]:
model.save_pretrained('./tokenizer')

In [ ]:
import gc
gc.collect()
torch.cuda.empty_cache()